In [1]:
import time
import datetime as dt
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
main_df = pd.read_csv('eq_database_place.csv')
dummy_eq = main_df.copy()

In [3]:
dummy_df = dummy_eq[dummy_eq['Place'].str.contains('US')]
# dummy_df = dummy_df.reindex(np.random.permutation(dummy_df.index))
dummy_df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,Place
123,04/16/1965,23:22:21,64.572,-160.375,Earthquake,15.0,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,ISCGEM857809,ISCGEM,ISCGEM,ISCGEM,Automatic,"Golovin, US"
131,04/26/1965,20:29:07,54.157,-162.590,Earthquake,36.8,NaN,NaN,5.6,MW,...,NaN,NaN,NaN,NaN,ISCGEM858049,ISCGEM,ISCGEM,ISCGEM,Automatic,"Aleutians East Borough, US"
136,04/29/1965,15:28:45,47.288,-122.406,Earthquake,64.7,NaN,NaN,6.7,MW,...,NaN,NaN,NaN,NaN,ISCGEM858143,ISCGEM,ISCGEM,ISCGEM,Automatic,"Tacoma, US"
138,05/01/1965,21:27:54,60.350,-146.176,Earthquake,15.0,NaN,NaN,5.6,MW,...,NaN,NaN,NaN,NaN,ISCGEM856572,ISCGEM,ISCGEM,ISCGEM,Automatic,"Cordova, US"
184,06/23/1965,11:09:17,56.543,-152.948,Earthquake,20.0,NaN,NaN,6.5,MW,...,NaN,NaN,NaN,NaN,ISCGEM856357,ISCGEM,ISCGEM,ISCGEM,Automatic,"Uhaiak (historical), US"


## NaN removing function

In [4]:
def nan_helper(y):
    """
    Helper to handle indices and logical indices of NaNs.
    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """
    
    return np.isnan(y), lambda z: z.nonzero()[0]

## String label encoding

In [5]:
def label_integer_encoder(my_df, series_name):
    """
    This function is encoding values of a series
    Parameter
    ---------
    * `my_df`: Pandas dataframe
    * `series_name`: Pandas series name to encode
    Returns : a encoded array
    """
    arr_name = np.array(list(my_df[str(series_name)]))
    label_arr_encoder = LabelEncoder()
    integer_arr_encoded = label_arr_encoder.fit_transform(arr_name)
    
    return integer_arr_encoded

## Interpolation function

In [6]:
def get_interpolation(my_df, nan_series):
    arr_series = np.array(my_df[str(nan_series)])
    nans, x = nan_helper(arr_series)
    arr_series[nans] = np.interp(x(nans), x(~nans), arr_series[~nans])
    return arr_series.round(2)

## Removing NaN values from the series

In [7]:
dummy_df['Depth Error'] = get_interpolation(dummy_df, 'Depth Error')
dummy_df['Depth Seismic Stations'] = get_interpolation(dummy_df, 'Depth Seismic Stations')
dummy_df['Magnitude Error'] = get_interpolation(dummy_df, 'Magnitude Error')
dummy_df['Magnitude Seismic Stations'] = get_interpolation(dummy_df, 'Magnitude Seismic Stations')
dummy_df['Azimuthal Gap'] = get_interpolation(dummy_df, 'Azimuthal Gap')
dummy_df['Horizontal Distance'] = get_interpolation(dummy_df, 'Horizontal Distance')
dummy_df['Horizontal Error'] = get_interpolation(dummy_df, 'Horizontal Error')
dummy_df['Root Mean Square'] = get_interpolation(dummy_df, 'Root Mean Square')

## Actual encoding of strings

In [8]:
dummy_df['Type'] = label_integer_encoder(dummy_df, 'Type')
dummy_df['Magnitude Type'] = label_integer_encoder(dummy_df, 'Magnitude Type')
dummy_df['Place'] = label_integer_encoder(dummy_df, 'Place')
dummy_df['Status'] = label_integer_encoder(dummy_df, 'Status')

## Dropping unwanted

In [9]:
dummy_df = dummy_df.drop(['ID', 'Source', 'Location Source', 'Magnitude Source'], axis=1)

## Time object numerical values

In [10]:
timestamp = []
for d, t in zip(dummy_df['Date'], dummy_df['Time']):
    try:
        ts = dt.datetime.strptime(d + ' ' + t, '%m/%d/%Y %H:%M:%S')
        timestamp.append(time.mktime(ts.timetuple())) # inverse funtion of localtime
    except ValueError as e:
        timestamp.append('ValueError')

time_s = pd.Series(timestamp)
dummy_df['TimeStamp'] = time_s.values
dummy_df = dummy_df.drop(['Date', 'Time'], axis=1)

In [11]:
dummy_df.head()

,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,Magnitude Error,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,Status,Place,TimeStamp
123,64.572,-160.375,0,15.0,31.61,16.0,6.0,5,0.24,10.0,261.0,1.48,99.0,0.86,0,55,-148630059.0
131,54.157,-162.590,0,36.8,31.61,16.0,5.6,5,0.24,10.0,261.0,1.48,99.0,0.86,0,4,-147776453.0
136,47.288,-122.406,0,64.7,31.61,16.0,6.7,5,0.24,10.0,261.0,1.48,99.0,0.86,0,123,-147535275.0
138,60.350,-146.176,0,15.0,31.61,16.0,5.6,5,0.24,10.0,261.0,1.48,99.0,0.86,0,33,-147340926.0
184,56.543,-152.948,0,20.0,31.61,16.0,6.5,5,0.24,10.0,261.0,1.48,99.0,0.86,0,129,-142798843.0


## Split into two

In [12]:
X = dummy_df[['Latitude', 'Longitude', 'Magnitude Error', 'Magnitude Type', 'Depth Error', 
              'Azimuthal Gap', 'Horizontal Distance', 'Horizontal Error', 'Root Mean Square']]
y = dummy_df[['Magnitude', 'Depth']]

## Train & Test Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fitting the model -- __RandomForestRegressor__

In [14]:
reg = RandomForestRegressor()
reg.fit(X_train, y_train)
preds = reg.predict(X_test)
print(preds)

[[  5.912   10.3815]
 [  6.11    21.72  ]
 [  5.88    36.34  ]
 [  5.987    9.836 ]
 [  5.69    23.49  ]
 [  5.843    8.1935]
 [  5.84    32.192 ]
 [  5.89   112.95  ]
 [  5.91    33.85  ]
 [  5.71    71.21  ]
 [  6.      11.39  ]
 [  6.061    9.7728]
 [  5.6     39.01  ]
 [  5.97    36.68  ]
 [  5.71    42.52  ]
 [  6.02    32.36  ]
 [  5.87    67.43  ]
 [  5.81    29.24  ]
 [  5.872    7.8313]
 [  5.97    39.471 ]
 [  6.201   10.098 ]
 [  5.688    8.1358]
 [  5.866    6.7558]
 [  5.95    40.54  ]
 [  5.88    22.999 ]
 [  5.91   124.98  ]
 [  5.734    6.723 ]
 [  6.      37.16  ]
 [  5.95    50.04  ]
 [  5.66    56.1   ]
 [  5.8     31.1   ]
 [  5.87    54.65  ]
 [  5.77    29.3   ]
 [  5.66    31.2   ]
 [  5.67    36.2   ]
 [  5.73    12.12  ]
 [  5.9     31.07  ]
 [  5.87    10.2   ]
 [  5.64    50.    ]
 [  5.74    35.46  ]
 [  5.848    9.6443]
 [  5.78    23.39  ]
 [  5.85    17.42  ]
 [  5.97    28.1   ]
 [  5.73    42.73  ]
 [  5.96    39.13  ]
 [  6.05    30.39  ]
 [  5.684    

In [15]:
accuracy = reg.score(X_test, y_test)
print(accuracy)

0.7774661713138058


## Fitting the model -- __GridSearchCV__

In [16]:
parameters = {'n_estimators' : [13, 18, 43, 77, 45, 450]}
gs = GridSearchCV(reg, parameters)

In [17]:
grid_fit = gs.fit(X_train, y_train)
best_fit = grid_fit.best_estimator_
gs_preds = best_fit.predict(X_test)
print(gs_preds)

[[  5.97571429  10.20233766]
 [  6.14025974  25.32597403]
 [  5.92207792  38.13766234]
 [  5.99493506   7.57977922]
 [  5.71298701  30.04155844]
 [  5.71064935   6.91209091]
 [  5.86493506  30.15402597]
 [  5.85844156 107.9961039 ]
 [  5.92077922  34.34415584]
 [  5.64675325  65.76233766]
 [  6.15532468  13.04709091]
 [  5.9238961    8.84696104]
 [  5.67792208  32.24545455]
 [  5.81818182  37.71558442]
 [  5.77532468  45.91636364]
 [  6.22857143  32.84675325]
 [  5.85194805  73.46077922]
 [  5.94545455  30.64779221]
 [  6.03792208  11.74112987]
 [  5.98051948  37.18662338]
 [  6.13207792   8.78933766]
 [  5.64649351   8.6177013 ]
 [  5.89922078   9.52542857]
 [  5.95324675  35.88441558]
 [  5.86493506  31.82116883]
 [  5.88961039 120.36233766]
 [  5.91142857   5.16554545]
 [  6.02207792  37.25506494]
 [  5.8987013   39.37792208]
 [  5.72337662  54.48571429]
 [  5.81428571  32.16103896]
 [  5.78701299  48.87532468]
 [  5.82077922  35.18467532]
 [  5.75584416  35.38077922]
 [  5.62857143

In [18]:
gs_accuracy = best_fit.score(X_test, y_test)
print(gs_accuracy)

0.7878061462506243
